# Interactive Demonstrations

**Live parameter tuning and visualization**

Use Jupyter widgets for real-time interaction!

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt
from primal_logic.hand import RoboticHand
from primal_logic.field import CoherenceField

try:
    from ipywidgets import interact, FloatSlider, IntSlider
    WIDGETS_AVAILABLE = True
except ImportError:
    print('⚠️  Install ipywidgets for interactive controls')
    print('pip install ipywidgets')
    WIDGETS_AVAILABLE = False

if WIDGETS_AVAILABLE:
    @interact(
        alpha=FloatSlider(min=0.3, max=0.8, step=0.02, value=0.54, description='Alpha'),
        beta=FloatSlider(min=0.0, max=0.5, step=0.05, value=0.1, description='Beta'),
        steps=IntSlider(min=100, max=1000, step=100, value=500, description='Steps')
    )
    def interactive_hand_simulation(alpha, beta, steps):
        hand = RoboticHand(alpha=alpha, beta=beta)
        field = CoherenceField()
        desired = np.ones((5, 3)) * 1.0
        
        angles_hist = []
        coherence_hist = []
        
        for i in range(steps):
            hand.step(desired)
            field.step(hand.get_angles())
            angles_hist.append(np.mean(hand.get_angles()))
            coherence_hist.append(field.get_coherence())
        
        time = np.arange(steps) * 0.01
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        ax1.plot(time, angles_hist, linewidth=2)
        ax1.axhline(y=1.0, color='r', linestyle='--')
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel('Mean Angle (rad)')
        ax1.set_title(f'Hand Closure (α={alpha}, β={beta})', fontweight='bold')
        ax1.grid(True, alpha=0.3)
        
        ax2.plot(time, coherence_hist, linewidth=2, color='green')
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Coherence')
        ax2.set_title('System Coherence', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f'Final angle: {angles_hist[-1]:.4f} rad')
        print(f'Final coherence: {coherence_hist[-1]:.6f}')
else:
    print('Interactive widgets not available')